In [145]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import max_error
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.svm import SVC

In [103]:
df = pd.read_csv('Final_Data.csv')
df_team = pd.read_csv('Final_Data_ByTeam.csv')

In [104]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Date,Home_Team,Away_Team,Field_Goal,Field_Goal_Attempts,Field_Goal_%,2Point,2Point_Attempts,...,Away_3Point_%,Away_Free_Throw,Away_Free_Throw_Attempt,Away_Free_Throw_%,Away_Points,Offensive_Efficiency,Away_Offensive_Efficiency,Point_Diff,O_Eff_Per_Points,Away_O_Eff_Per_Points
0,0,0,2018-10-16,BOS,PHI,42,97,0.433,31,60,...,0.192,14,23,0.609,87,108.247423,100.000000,18,1.030928,1.149425
1,1,1,2018-10-16,GSW,OKC,42,95,0.442,35,69,...,0.270,24,37,0.649,100,113.684211,109.890110,8,1.052632,1.098901
2,2,2,2018-10-17,CHO,MIL,41,92,0.446,25,54,...,0.412,15,20,0.750,113,121.739130,132.941176,-1,1.086957,1.176471
3,3,3,2018-10-17,DET,BRK,39,92,0.424,33,68,...,0.185,15,22,0.682,100,111.956522,121.951220,3,1.086957,1.219512
4,4,4,2018-10-17,HOU,NOP,39,92,0.424,23,44,...,0.400,17,22,0.773,131,121.739130,133.673469,-19,1.086957,1.020408


In [105]:
df_team_data = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Free_Throw_%','Away_Free_Throw_%','2Point_%','3Point_%','Away_2Point_%','Away_3Point_%','Offensive_Efficiency','Away_Offensive_Efficiency','O_Eff_Per_Points','Away_O_Eff_Per_Points','Points','Away_Points']

In [106]:
#Final Dataset I will be using
df = df[df_team_data]

In [107]:
#checkSum of nulls
df.isnull().sum()

Home_Team                    0
Away_Team                    0
Field_Goal_%                 0
Away_Field_Goal_%            0
Free_Throw_%                 0
Away_Free_Throw_%            0
2Point_%                     0
3Point_%                     0
Away_2Point_%                0
Away_3Point_%                0
Offensive_Efficiency         0
Away_Offensive_Efficiency    0
O_Eff_Per_Points             0
Away_O_Eff_Per_Points        0
Points                       0
Away_Points                  0
dtype: int64

In [108]:
df.head()
#df = df.iloc[: , 1:]


,Home_Team,Away_Team,Field_Goal_%,Away_Field_Goal_%,Free_Throw_%,Away_Free_Throw_%,2Point_%,3Point_%,Away_2Point_%,Away_3Point_%,Offensive_Efficiency,Away_Offensive_Efficiency,O_Eff_Per_Points,Away_O_Eff_Per_Points,Points,Away_Points
0,BOS,PHI,0.433,0.391,0.714,0.609,0.517,0.297,0.475,0.192,108.247423,100.000000,1.030928,1.149425,105,87
1,GSW,OKC,0.442,0.363,0.944,0.649,0.507,0.269,0.426,0.270,113.684211,109.890110,1.052632,1.098901,108,100
2,CHO,MIL,0.446,0.494,0.636,0.750,0.463,0.421,0.549,0.412,121.739130,132.941176,1.086957,1.176471,112,113
3,DET,BRK,0.424,0.488,0.864,0.682,0.485,0.250,0.636,0.185,111.956522,121.951220,1.086957,1.219512,103,100
4,HOU,NOP,0.424,0.531,0.750,0.773,0.523,0.333,0.575,0.400,121.739130,133.673469,1.086957,1.020408,112,131


In [109]:
#Split data in training set (80%)& test set (20%)

def split_data(df, drop_list):
    df = df.drop(drop_list,axis=1)
    print(df.columns)
    #test train split time
    from sklearn.model_selection import train_test_split
    y = df['Points'].values #target
    X = df.drop(['Points'],axis=1).values #features
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

    print("train-set size: ", len(y_train),
      "\ntest-set size: ", len(y_test))
    return X_train, X_test, y_train, y_test

In [110]:
#Below is funtion to define classifier and get predictions. 
def get_predictions(clf, X_train, y_train, X_test):
    # create classifier
    clf = clf
    # fit it to training data
    clf.fit(X_train,y_train)
    # predict using test data
    y_pred = clf.predict(X_test)
    return y_pred

In [111]:
#Function to print the regressor's scores
def print_scores(y_test,y_pred):
    print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))
    print('R2: ', r2_score(y_test, y_pred))
    print('Max Error: ', max_error(y_test, y_pred))

In [117]:
# RandomForest #1 - All Data in 
drop_list = ['Home_Team','Away_Team','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(RandomForestRegressor(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Field_Goal_%', 'Free_Throw_%', '2Point_%', '3Point_%',
       'Offensive_Efficiency', 'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  0.443560830860534
R2:  0.987435368156364
Max Error:  17.169999999999987


In [135]:
# RandomForest #1.5 Only FG% & FT%, as well OFF_EFF
drop_list = ['Home_Team','Away_Team','Away_Field_Goal_%','Away_Free_Throw_%','2Point_%','Away_2Point_%','3Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(RandomForestRegressor(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Field_Goal_%', 'Free_Throw_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  0.40764094955489616
R2:  0.9883625162763849
Max Error:  15.789999999999992


In [118]:
#RandomForest #2 Takes Away Field Goal % *****************8
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']

X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(RandomForestRegressor(n_estimators=100, max_depth=3), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  3.9229708288908944
R2:  0.7865751167659917
Max Error:  38.33782619963111


In [124]:
#RandomForest #3 Takes AWay FG%, FT%
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Free_Throw_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(RandomForestRegressor(n_estimators=100, max_depth=3), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['2Point_%', '3Point_%', 'Offensive_Efficiency', 'O_Eff_Per_Points',
       'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  3.9524944341472614
R2:  0.7872299733591269
Max Error:  37.86290907010678


In [125]:
#RandomForest #4 Takes Away FG%, FT%, 2%
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Free_Throw_%','Away_Free_Throw_%','2Point_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(RandomForestRegressor(n_estimators=100, max_depth=3), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['3Point_%', 'Offensive_Efficiency', 'O_Eff_Per_Points', 'Points',
       'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  3.951686345909902
R2:  0.7875278287197957
Max Error:  38.22955914594587


In [130]:
#RandomForest #5 Takes Away FG%, FT%, 2%,3%
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Free_Throw_%','Away_Free_Throw_%','2Point_%','Away_2Point_%','3Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']

X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(RandomForestRegressor(n_estimators=100, max_depth=3), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Offensive_Efficiency', 'O_Eff_Per_Points', 'Points', 'Away_Points'], dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  3.9000733082792376
R2:  0.789770872831713
Max Error:  38.17303050929985


In [131]:
#RandomForest #6 Takes Away FG%, FT%, 2%,3%,Off_eff
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Free_Throw_%','Away_Free_Throw_%','2Point_%','Away_2Point_%','3Point_%','Away_3Point_%','Offensive_Efficiency','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']

X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(RandomForestRegressor(n_estimators=100, max_depth=3), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['O_Eff_Per_Points', 'Points', 'Away_Points'], dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  8.846132569824425
R2:  0.15641455046254504
Max Error:  45.73765673858999


In [ ]:
#Logistic Regression
drop_list = ['Home_Team','Away_Team','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(LogisticRegression(max_iter=10), X_train, y_train, X_test)
print_scores(y_test,y_pred)

In [ ]:
#Logistic Regression#2
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']

X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(LogisticRegression(max_iter=10), X_train, y_train, X_test)
print_scores(y_test,y_pred)

In [133]:
#GaussianNB #1 
drop_list = ['Home_Team','Away_Team','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(GaussianNB(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Field_Goal_%', 'Free_Throw_%', '2Point_%', '3Point_%',
       'Offensive_Efficiency', 'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  6.155786350148368
R2:  0.5730227747024581
Max Error:  37


In [137]:
#GNB 1.5 Takes Away Field Goal % *****************
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']

X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(GaussianNB(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  5.4347181008902075
R2:  0.6621008321410449
Max Error:  26


In [134]:
#GNB2 Takes Away Field Goal %
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(GaussianNB(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  5.4347181008902075
R2:  0.6621008321410449
Max Error:  26


In [140]:
#KNN Not Good
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(KNeighborsRegressor(n_neighbors=1500), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  7.179065281899111
R2:  0.39440730573001137
Max Error:  45.282


In [142]:
#DecisionTree Not Good
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(DecisionTreeRegressor(max_depth=3), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  5.24182645343869
R2:  0.6986861620122093
Max Error:  36.307317073170736


In [144]:
#Ridge TooGood? - Pass
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(Ridge(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  0.8028711186465648
R2:  0.9878691431229548
Max Error:  14.214754353226425


In [146]:
 #SVC - Pass
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(SVC(max_iter=2), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674


/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Mean Absolute Error:  8.732937685459941
R2:  0.1781959743530288
Max Error:  43


In [149]:
#XGB - Pass
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 3, alpha = 1, n_estimators = 20), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
[15:14:32] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error:  14.160300484043201
R2:  -0.8284740316390578
Max Error:  53.206756591796875


In [150]:
#LinearRegression - Pass
drop_list = ['Home_Team','Away_Team','Field_Goal_%','Away_Field_Goal_%','Away_Free_Throw_%','Away_2Point_%','Away_3Point_%','Away_Offensive_Efficiency','Away_O_Eff_Per_Points']
X_train, X_test, y_train, y_test = split_data(df, drop_list)
y_pred = get_predictions(LinearRegression(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['Free_Throw_%', '2Point_%', '3Point_%', 'Offensive_Efficiency',
       'O_Eff_Per_Points', 'Points', 'Away_Points'],
      dtype='object')
train-set size:  2695 
test-set size:  674
Mean Absolute Error:  0.7132216062627016
R2:  0.9901260047496875
Max Error:  13.054499242375499
